## Cleaning Data from Quebec Weather Stations

## Connect and set up SQL 

In [3]:
import sqlite3
con = sqlite3.connect("WEATHER.db") #create new database called Weather or connect to it if it already exists

cursor_obj = con.cursor() #create a cursor to fetch results from SQL queries

In [4]:
!pip install "sqlalchemy<2"

#allows use of sql magic command
%load_ext sql
#connects to database
%sql sqlite:///WEATHER.db


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @WEATHER.db'

## Load data using Pandas

In [15]:
import csv, pandas as pd, numpy as np

df1 = pd.read_csv("/Users/maggiesullens/Library/Mobile Documents/com~apple~CloudDocs/Spruce Budworm/Retrieved weather/qc_weather.csv")
df1.rename(columns={"Unnamed: 0": "ID"}, inplace=True)
df1.to_sql("QC_WEATHER", con, if_exists='replace', index=False)


7805150

## Show glipse of data

In [18]:
%%sql 
SELECT *
FROM QC_WEATHER LIMIT 3;


 * sqlite:///WEATHER.db
Done.


ID,prov,station_name,station_id,lat,lon,elev,date,max_temp,mean_temp,min_temp,total_precip
1,QC,DALHOUSIE STATION,5199,45.3,-74.47,70.0,1974-09-01,21.1,13.4,5.6,0.0
2,QC,DALHOUSIE STATION,5199,45.3,-74.47,70.0,1974-09-02,16.1,13.1,10.0,8.6
3,QC,DALHOUSIE STATION,5199,45.3,-74.47,70.0,1974-09-03,15.6,12.5,9.4,3.3


## Assign each day a group number
This is to bunch all days together in a new dataset so that we can find the averages for each day.
We complete this using pandas and in batches to reduce run time. 

In [83]:
batch_size = 100000
offset = 0
all_data = []

while True:
    query = f"""
    SELECT *,
           DENSE_RANK() OVER (ORDER BY date) AS date_group
    FROM (
        SELECT *
        FROM QC_WEATHER
        ORDER BY date
        LIMIT {batch_size} OFFSET {offset}
    ) AS DATE_GROUPS;
    """
    
    result = %sql $query
    df = result.DataFrame()
    
    if df.empty:
        break
    
    all_data.append(df)
    offset += batch_size

final_df = pd.concat(all_data, ignore_index=True)

 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:///WEATHER.db
Done.
 * sqlite:

## Convert back to SQL

In [84]:
final_df.to_sql("DATE_GROUPS", con, if_exists='replace', index=False)

7805150

## Glimpse of New Dataset

In [85]:
%%sql
SELECT * FROM DATE_GROUPS LIMIT 3;

 * sqlite:///WEATHER.db
Done.


ID,prov,station_name,station_id,lat,lon,elev,date,max_temp,mean_temp,min_temp,total_precip,date_group
1863303,QC,MONTREAL MCGILL,5420,45.5,-73.58,56.9,1871-07-01,28.3,20.8,13.3,0.0,1
1863304,QC,MONTREAL MCGILL,5420,45.5,-73.58,56.9,1871-07-02,26.1,22.8,19.4,0.0,2
1863305,QC,MONTREAL MCGILL,5420,45.5,-73.58,56.9,1871-07-03,28.9,20.3,11.7,0.0,3


## Make Daily Averages Dataset for Max Temp, Min Temp, Mean Temp, and Total Precip

In [86]:
%%sql
DROP TABLE IF EXISTS QC_DAY_AVERAGE_WEATHER;

CREATE TABLE QC_DAY_AVERAGE_WEATHER AS
SELECT date_group, date, ROUND(AVG(max_temp),4) as avg_max_temp, 
ROUND(AVG(min_temp),4) as avg_min_temp, ROUND(AVG(mean_temp),4) as avg_mean_temp,
ROUND(AVG(total_precip),4) as avg_total_precip FROM DATE_GROUPS GROUP BY date_group;

SELECT * FROM QC_DAY_AVERAGE_WEATHER LIMIT 3;

 * sqlite:///WEATHER.db
Done.
Done.
Done.


date_group,date,avg_max_temp,avg_min_temp,avg_mean_temp,avg_total_precip
1,1871-07-01,7.5856,-3.3965,2.0994,2.4766
2,1871-07-02,8.4455,-2.3483,3.0549,2.9306
3,1871-07-03,8.1727,-2.2651,2.9604,2.7102


## Check to see if there are any duplicate days

In [87]:
%%sql
SELECT date_group, date, COUNT(*) AS occurrences
FROM QC_DAY_AVERAGE_WEATHER
GROUP BY date
HAVING COUNT(*) > 1;

 * sqlite:///WEATHER.db
Done.


date_group,date,occurrences


## Save Average Dataset to CSV

In [88]:
query = "SELECT * FROM QC_DAY_AVERAGE_WEATHER"
data_frame = pd.read_sql(query, con)

csv_file_name = 'QC_daily_average_data.csv'
data_frame.to_csv(csv_file_name, index=False)
print(f"Data exported to '{csv_file_name}' successfully.")

Data exported to 'QC_daily_average_data.csv' successfully.


## Make Yearly Averages Dataset of the Daily Averages for Max Temp, Min Temp, Mean Temp, and Total Precip

In [89]:
query = f"""
SELECT *,
    DENSE_RANK() OVER (ORDER BY strftime('%Y', date)) AS year_group
FROM (
    SELECT *
    FROM QC_DAY_AVERAGE_WEATHER
    ORDER BY date
) AS subquery_year_group;
"""

result2 = %sql $query
df2 = result2.DataFrame()

# Now save to a new or same table
df2.to_sql("YEAR_GROUPS", con, if_exists='replace', index=False)

 * sqlite:///WEATHER.db
Done.


12387

In [97]:
%%sql
DROP TABLE IF EXISTS QC_YEARLY_DAY_AVERAGES_WEATHER;

CREATE TABLE QC_YEARLY_DAY_AVERAGES_WEATHER AS
SELECT year_group, date, ROUND(AVG(avg_max_temp),2) as year_avg_max_temp, 
ROUND(AVG(avg_min_temp),2) as year_avg_min_temp, ROUND(AVG(avg_mean_temp),2) as year_avg_mean_temp,
ROUND(AVG(avg_total_precip),2) as year_avg_total_precip FROM YEAR_GROUPS GROUP BY year_group;

SELECT * FROM QC_YEARLY_DAY_AVERAGES_WEATHER LIMIT 3;

 * sqlite:///WEATHER.db
Done.
Done.
Done.


year_group,date,year_avg_max_temp,year_avg_min_temp,year_avg_mean_temp,year_avg_total_precip
1,1871-07-01,9.29,-1.2,4.05,2.82
2,1872-01-01,8.51,-1.89,3.32,2.74
3,1873-01-01,8.15,-2.29,2.93,2.58


In [96]:
query = "SELECT * FROM QC_YEARLY_DAY_AVERAGES_WEATHER"
data_frame = pd.read_sql(query, con)

csv_file_name = 'QC_yearly_daily_averages_data.csv'
data_frame.to_csv(csv_file_name, index=False)
print(f"Data exported to '{csv_file_name}' successfully.")

Data exported to 'QC_yearly_daily_averages_data.csv' successfully.


In [ ]:
%%sql
SELECT COUNT(